#### 2.预测模型 3.扩展数据集(ccks+FN) 4.扩展数据集用于预训练(ccks+FN) 5.验证(四川的500条上) 

In [ ]:
# 2.训练出ccks的预测模型
from CC.trainer import NERTrainer

args = {
    'num_epochs': 30,
    'num_gpus': [0, 1, 2, 3],
    'bert_config_file_name': './model/chinese_wwm_ext/bert_config.json',
    'pretrained_file_name': './model/chinese_wwm_ext/pytorch_model.bin',
    'hidden_dim': 300,
    'max_seq_length': 150,
    'max_scan_num': 1000000,
    'train_file': 'data/ccks/train-json.csv',
    # 'eval_file': './data/ccks/dev.csv',
    # 'test_file': './data/ccks/test.csv',
    'eval_file': './data/FN/sc-json/dev.csv',
    'test_file': './data/FN/sc-json/test.csv',
    'bert_vocab_file': './model/chinese_wwm_ext/vocab.txt',
    'tag_file': 'data/ccks/ccks_tags_list.txt',
    'output_eval': True,
    'loader_name': 'le_loader',
    "word_embedding_file": "./data/tencent/word_embedding.txt",
    "word_vocab_file": "./data/tencent/tencent_vocab.txt",
    "default_tag": "O",
    'batch_size': 32,
    'eval_batch_size': 64,
    'do_shuffle': True,
    'model_name': 'LEBert',
    'task_name': 'ccks_predict_model'
}

# Trainer
trainer = NERTrainer(**args)

for i in trainer(lr2=1e-2):
    a = i

In [ ]:
# 3.预测四川数据集（多标签）
from CC.predicter import NERPredict
import json

# 使用了预训练模型
args["lstm_crf_model_file"] = "save_model/ccks_predict_model/lstm_crf/lstm_crf_1320.pth"
args["bert_model_file"] = "save_model/ccks_predict_model/LEBert/LEBert_1320.pth"
predict = NERPredict(**args)

filename = "data/FN/sc-super/sc-train_400.json"

batch_size = 64
index = 0
sentences = []

with open("data/FN/sc-super/sc_super_400.json", "w", encoding="utf-8") as out:
    with open(filename, "r", encoding="utf-8") as f:
        for line in f:
            data = json.loads(line)
            text = data["text"]
            
            sentences.append(text)
            index += 1
            if index % batch_size == batch_size-1:
                for s, label in predict(sentences):
                    
                    assert len(s[:args["max_seq_length"]-2])==len(label)
                    out.write(f"""{json.dumps({"text":s[:args["max_seq_length"]-2],"label":label},ensure_ascii=False)}\n""")
                sentences = []
                out.flush()
        if len(sentences)>0:
            for s, label in predict(sentences):
                assert len(s[:args["max_seq_length"]])==len(label)
                out.write(f"""{json.dumps({"text":s[:args["max_seq_length"]-2],"label":label},ensure_ascii=False)}\n""")

In [ ]:
# Bert预训练
args = {
    'num_epochs': 35,
    'num_gpus': [0, 1, 2, 3],
    'bert_config_file_name': './model/chinese_wwm_ext/bert_config.json',
    # 'pretrained_file_name': './model/chinese_wwm_ext/pytorch_model.bin',
    'pretrained_file_name': './save_pretrained/FN_multiple_pretrained_2/Bert_8960/pytorch_model.bin',
    'max_seq_length': 512,
    'max_scan_num': 1000000,
    'train_file': './data/FN/sc-super/train_super.json',
    'eval_file': './data/FN/sc-json-500/dev.json',
    'test_file': './data/FN/sc-json-500/test.json',
    'bert_vocab_file': './model/chinese_wwm_ext/vocab.txt',
    'tag_file': 'data/ccks/ccks+FN_tags_list.txt',
    'loader_name': 'ptloader_v2',
    "word_embedding_file": "./data/tencent/word_embedding.txt",
    "word_vocab_file": "./data/tencent/tencent_vocab.txt",
    "word_vocab_file_with_tag": "./data/tencent/tencent_vocab_with_tag.json",
    "default_tag": "O",
    'batch_size': 8,
    'eval_batch_size': 32,
    'pass_none_rule': True,
    'skip_single_matched_word': True,
    'do_shuffle': True,
    'task_name': 'FN_multiple_pretrained_3',
    "use_gpu": True,
    "debug": True,
    "tag_rules": {
        "O": "非实体",
        "KEYWORD": "关键词",
        "DIS": "疾病或诊断",
        "ANA": "解剖部位",
        "LAB": "实验室检验",
        "MED": "药物",
        "OPE": "手术",
        "IMA": "影像检查",
    }
}

from CC.pre_trained import NERPreTrainer
pre_trainer = NERPreTrainer(**args)

for i in pre_trainer(lr=1e-5):
    a = i

In [ ]:
#5.验证2
from CC.trainer import NERTrainer
args = {
    'num_epochs': 30,
    'num_gpus': [0, 1, 2, 3],
    'bert_config_file_name': './model/chinese_wwm_ext/bert_config.json',
    # 'pretrained_file_name': './model/chinese_wwm_ext/pytorch_model.bin',
    'pretrained_file_name': './save_pretrained/FN_multiple_pretrained_3/Bert_33600/pytorch_model.bin',

    'hidden_dim': 300,
    'max_seq_length': 512,
    'max_scan_num': 1000000,
    # 'train_file': './data/ccks/train.json',
    # 'eval_file': './data/ccks/sc-json-500/dev.json',
    # 'test_file': './data/ccks/sc-json-500/test.json',
    'train_file': './data/FN/fj+sc/train(400).json',
    'eval_file': './data/FN/sc-json-500/dev.json',
    'test_file': './data/FN/sc-json-500/test.json',
    'bert_vocab_file': './model/chinese_wwm_ext/vocab.txt',
    'tag_file': './data/FN/tags_list.txt',
    'loader_name': 'le_loader',
    'output_eval':True,
    "word_embedding_file":"./data/tencent/word_embedding.txt",
    "word_vocab_file":"./data/tencent/tencent_vocab.txt",
    # "word_vocab_file_with_tag": "./data/tencent/tencent_vocab_with_tag.json",
    "default_tag":"O",
    'batch_size': 4,
    'eval_batch_size': 64,
    'do_shuffle': True,
    "use_gpu": True,
    "debug": True,
    'model_name': 'LEBert',
    'task_name': 'FN_dev_3'
}
trainer = NERTrainer(**args)

for i in trainer():
    a = i

In [ ]:
from CC.trainer import NERTrainer
args = {
    'num_epochs': 30,
    'num_gpus': [0, 1, 2, 3],
    'bert_config_file_name': './model/chinese_wwm_ext/bert_config.json',
    'pretrained_file_name': './model/chinese_wwm_ext/pytorch_model.bin',
    'hidden_dim': 300,
    'max_seq_length': 512,
    'max_scan_num': 1000000,
    'train_file': './data/FN/fj+sc/train(400).json',
    'eval_file': './data/FN/sc-json/dev.csv',
    'test_file': './data/FN/sc-json/test.csv',
    'bert_vocab_file': './model/chinese_wwm_ext/vocab.txt',
    'tag_file': './data/FN/tags_list.txt',
    'loader_name': 'le_loader',
    'output_eval':True,
    "word_embedding_file":"./data/tencent/word_embedding.txt",
    "word_vocab_file":"./data/tencent/tencent_vocab.txt",

    "default_tag":"O",
    'batch_size': 4,
    'eval_batch_size': 64,
    'do_shuffle': True,
    "use_gpu": True,
    "debug": True,
    'model_name': 'LEBert',
    'task_name': 'FN_dev_base_2'
}
trainer = NERTrainer(**args)

for i in trainer():
    a = i

#### 1预训练(ccks+FJ)   2.预测模型 3.扩展数据集(ccks+FN) 4.扩展数据集用于预训练(ccks+FN) 5.验证(四川的500条上) 


In [ ]:
# 1.预训练
args = {
    'num_epochs': 35,
    'num_gpus': [0, 1, 2, 3],
    'bert_config_file_name': './model/chinese_wwm_ext/bert_config.json',
    # 'pretrained_file_name': './model/chinese_wwm_ext/pytorch_model.bin',
    'pretrained_file_name': './save_pretrained/ccks_pretrained/Bert_3160/pytorch_model.bin',
    'max_seq_length': 512,
    'max_scan_num': 1000000,
    'train_file': './data/ccks/ccks/train_all.json',
    'eval_file': './data/ccks/dev.json',
    'test_file': './data/ccks/test.json',
    'bert_vocab_file': './model/chinese_wwm_ext/vocab.txt',
    'tag_file': 'data/ccks/ccks_tags_list.txt',
    'loader_name': 'ptloader_v2',
    "word_embedding_file": "./data/tencent/word_embedding.txt",
    "word_vocab_file": "./data/tencent/tencent_vocab.txt",
    "word_vocab_file_with_tag": "./data/tencent/tencent_vocab_with_tag.json",
    "default_tag": "O",
    'batch_size': 8,
    'eval_batch_size': 32,
    'pass_none_rule': True,
    'skip_single_matched_word': True,
    'do_shuffle': True,
    'task_name': 'ccks_pretrained_2',
    "use_gpu": True,
    "debug": True,
    "tag_rules": {
        "O": "非实体",
        "KEYWORD":"异常关键词",
        "DIS": "疾病或诊断",
        "ANA": "解剖部位",
        "LAB": "实验室检验",
        "MED": "药物",
        "OPE": "手术",
        "IMA": "影像检查",
    }
}

from CC.pre_trained import NERPreTrainer
pre_trainer = NERPreTrainer(**args)

for i in pre_trainer():
    a = i

In [ ]:
# 2.预测 修改：训练集要替换成ccks，预测ccks标签+keyword
from CC.trainer import NERTrainer
args = {
    'num_epochs': 30,
    'num_gpus': [0, 1, 2, 3],
    'bert_config_file_name': './model/chinese_wwm_ext/bert_config.json',
    # 'pretrained_file_name': './model/chinese_wwm_ext/pytorch_model.bin',
    'pretrained_file_name': './save_pretrained/ccks_pretrained/Bert_6055/pytorch_model.bin', #第一次的 只有ccks的预训练
    # 'pretrained_file_name': './save_pretrained/ccks_pretrained/Bert_6055/pytorch_model.bin',
    'hidden_dim': 300,
    'max_seq_length': 512,
    'max_scan_num': 1000000,
    'train_file': './data/ccks/ccks+FN_FJ/train.json',
    'eval_file': './data/FN/sc-json-500/dev.json',
    'test_file': './data/FN/sc-json-500/test.json',
    'bert_vocab_file': './model/chinese_wwm_ext/vocab.txt',
    'tag_file': 'data/ccks/ccks+FN_tags_list.txt',
    'loader_name': 'le_loader',
    'output_eval':True,
    "word_embedding_file":"./data/tencent/word_embedding.txt",
    "word_vocab_file":"./data/tencent/tencent_vocab.txt",
    "default_tag":"O",
    'batch_size': 16,
    'eval_batch_size': 32,
    'do_shuffle': True,
    "use_gpu": True,
    "debug": True,
    'model_name': 'LEBert',
    'task_name': 'FN+CCKS_predict_model'
}
# trainer = NERTrainer(**args)

# for i in trainer():
#     a = i




In [ ]:
# 扩展数据集   修改：要扩展ccks标签+FN
from CC.predicter import NERPredict
import json

# 使用了预训练模型
args["lstm_crf_model_file"] = "save_model/FN+CCKS_predict_model/lstm_crf/lstm_crf_.pth"
args["bert_model_file"] = "save_model/FN+CCKS_predict_model/LEBert/LEBert_.pth"
predict = NERPredict(**args)

filename = "data/FN/fj+sc/train(400).json"

batch_size = 40
index = 0
sentences = []

with open("data/FN/sc_fj_tags_keyword/train(400)_ccks.json", "w", encoding="utf-8") as out:
    with open(filename, "r", encoding="utf-8") as f:
        for line in f:
            data = json.loads(line)
            text = data["text"]
            
            sentences.append(text)
            index += 1
            if index % batch_size == batch_size-1:
                for s, label in predict(sentences):
                    
                    assert len(s[:args["max_seq_length"]-2])==len(label)
                    out.write(f"""{json.dumps({"text":s[:args["max_seq_length"]-2],"label":label},ensure_ascii=False)}\n""")
                sentences = []
                out.flush()
        if len(sentences)>0:
            for s, label in predict(sentences):
                assert len(s[:args["max_seq_length"]])==len(label)
                out.write(f"""{json.dumps({"text":s[:args["max_seq_length"]-2],"label":label},ensure_ascii=False)}\n""")

In [ ]:
# 预训练 
# sc&fj_tags_ccks_pretrain:多标签的sc和fj数据集（各400条）进行预训练
# sc&fj_tags_ccks_pretrain_2:跟上面一样 lr=1e-5
# sc_fj_tags_keyword_pretrain:keyword标签的sc和fj数据集（各400条）进行预训练
args = {
    'num_epochs': 35,
    'num_gpus': [0, 1],
    'bert_config_file_name': './model/chinese_wwm_ext/bert_config.json',
    'pretrained_file_name': './model/chinese_wwm_ext/pytorch_model.bin',
    # 'pretrained_file_name': './save_pretrained/sc&fj_tags_ccks_pretrain/Bert_11640/pytorch_model.bin',
    'max_seq_length': 512,
    'max_scan_num': 1000000,
    'train_file': './data/FN/sc_fj_tags_keyword/fj_400_sc_400.json',
    'eval_file': './data/FN/sc_test100/test.json',
    'test_file': './data/FN/sc_test100/test.jsonn',
    'bert_vocab_file': './model/chinese_wwm_ext/vocab.txt',
    # 'tag_file': './data/ccks/ccks+FN_tags_list.txt',
    'tag_file': './data/FN/tags_list.txt',
    'loader_name': 'ptloader_v2',
    "word_embedding_file": "./data/tencent/word_embedding.txt",
    "word_vocab_file": "./data/tencent/tencent_vocab.txt",
    "word_vocab_file_with_tag": "./data/tencent/tencent_vocab_with_tag.json",
    "default_tag": "O",
    'batch_size': 8,
    'eval_batch_size': 32,
    'pass_none_rule': True,
    'skip_single_matched_word': True,
    'do_shuffle': True,
    'task_name': 'sc_fj_tags_keyword_pretrain',
    "use_gpu": True,
    "debug": True,
    "tag_rules": {
        "O": "非实体",
        "KEYWORD":"异常关键词",
        "DIS": "疾病或诊断",
        "ANA": "解剖部位",
        "LAB": "实验室检验",
        "MED": "药物",
        "OPE": "手术",
        "IMA": "影像检查",
    }
    
}

from CC.pre_trained import NERPreTrainer
pre_trainer = NERPreTrainer(**args)

for i in pre_trainer(lr=1e-4):
    a = i

In [ ]:
# 
from CC.trainer import NERTrainer
args = {
    'num_epochs': 30,
    'num_gpus': [0, 1],
    'bert_config_file_name': './model/chinese_wwm_ext/bert_config.json',
    # 'pretrained_file_name': './model/chinese_wwm_ext/pytorch_model.bin',
    'pretrained_file_name': './save_pretrained/sc&fj_tags_ccks_pretrain_2/Bert_10088/pytorch_model.bin',
    'hidden_dim': 300,
    'max_seq_length': 512,
    'max_scan_num': 1000000,
    # 'train_file': './data/FN/fj+sc/train(400).json',
    # 'train_file': './data/FN/sc_json_500/train.json',
    'train_file': './data/FN/fj+sc/train(400).json',
    'eval_file': './data/FN/sc_test100/test.json',
    'test_file': './data/FN/sc_test100/test.json',
    'bert_vocab_file': './model/chinese_wwm_ext/vocab.txt',
    'tag_file': './data/FN/tags_list.txt',
    'loader_name': 'le_loader',
    'output_eval':True,
    "word_embedding_file":"./data/tencent/word_embedding.txt",
    "word_vocab_file":"./data/tencent/tencent_vocab.txt",
    # "word_vocab_file_with_tag": "./data/tencent/tencent_vocab_with_tag.json",
    "default_tag":"O",
    'batch_size': 8,
    'eval_batch_size': 64,
    'do_shuffle': True,
    "use_gpu": True,
    "debug": True,
    'model_name': 'LEBert',
    'task_name': 'FN_fj_sc_tag_ccks_pre_2'
}
trainer = NERTrainer(**args)

for i in trainer(lr2=1e-2):
    a = i

#### 1.外部数据集训练一个预测模型 2.把福建和四川总共800条的数据集标签拓展成多标签的 
#### 3.把数据集预训练 4.验证

In [ ]:
# 2.预测 
from CC.trainer import NERTrainer
args = {
    'num_epochs': 30,
    'num_gpus': [0, 1],
    'bert_config_file_name': './model/chinese_wwm_ext/bert_config.json',
    'pretrained_file_name': './model/chinese_wwm_ext/pytorch_model.bin',
    # 'pretrained_file_name': './save_pretrained/ccks_pretrained_2/Bert_6055/pytorch_model.bin', 
    'hidden_dim': 300,
    'max_seq_length': 512,
    'max_scan_num': 1000000,
    'train_file': './data/ccks/train.json',
    'eval_file': './data/ccks/dev.json',
    'test_file': './data/ccks/test.json',
    'bert_vocab_file': './model/chinese_wwm_ext/vocab.txt',
    'tag_file': 'data/ccks/ccks+FN_tags_list.txt',
    'loader_name': 'le_loader',
    'output_eval':True,
    "word_embedding_file":"./data/tencent/word_embedding.txt",
    "word_vocab_file":"./data/tencent/tencent_vocab.txt",
    "default_tag":"O",
    'batch_size': 16,
    'eval_batch_size': 64,
    'do_shuffle': True,
    "use_gpu": True,
    "debug": True,
    'model_name': 'LEBert',
    'task_name': 'ccks_predict_model_2'
}
# trainer = NERTrainer(**args)

# for i in trainer():
#     a = i

In [ ]:
# 3.预测fj+sc数据集（多标签）
from CC.predicter import NERPredict
import json

# 使用了预训练模型
args["lstm_crf_model_file"] = "save_model/ccks_predict_model_2/lstm_crf/lstm_crf_690.pth"
args["bert_model_file"] = "save_model/ccks_predict_model_2/LEBert/LEBert_690.pth"


predict = NERPredict(**args)

filename = "data/FN/sc_fj_tags_keyword/fj_400_sc_400.json"

batch_size = 64
index = 0
sentences = []

with open("data/FN/sc_fj_tags_ccks/fj_400_sc_400_ccks.json", "w", encoding="utf-8") as out:
    with open(filename, "r", encoding="utf-8") as f:
        for line in f:
            data = json.loads(line)
            text = data["text"]
            
            sentences.append(text)
            index += 1
            if index % batch_size == batch_size-1:
                for s, label in predict(sentences):
                    
                    assert len(s[:args["max_seq_length"]-2])==len(label)
                    out.write(f"""{json.dumps({"text":s[:args["max_seq_length"]-2],"label":label},ensure_ascii=False)}\n""")
                sentences = []
                out.flush()
        if len(sentences)>0:
            for s, label in predict(sentences):
                assert len(s[:args["max_seq_length"]])==len(label)
                out.write(f"""{json.dumps({"text":s[:args["max_seq_length"]-2],"label":label},ensure_ascii=False)}\n""")

In [ ]:
# 预训练 
# sc&fj_tags_ccks_pretrain:多标签的sc和fj数据集（各400条）进行预训练  需要重新预训练
# sc&fj_tags_ccks_pretrain_2:跟上面一样 lr=1e-5
# sc_fj_tags_keyword_pretrain:keyword标签的sc和fj数据集（各400条）进行预训练


# os.environ['CUDA_VISIBLE_DEVICES']='2, 3'
args = {
    'num_epochs': 35,
    'num_gpus': [0, 1],
    'bert_config_file_name': './model/chinese_wwm_ext/bert_config.json',
    # 'pretrained_file_name': './model/chinese_wwm_ext/pytorch_model.bin',
    'pretrained_file_name': './save_pretrained/sc_fj_tags_keyword_pretrain/Bert_12080/pytorch_model.bin',
    'max_seq_length': 512,
    'max_scan_num': 1000000,
    'train_file': './data/FN/sc_fj_tags_keyword/fj_400_sc_400.json',
    'eval_file': './data/FN/sc_test100/test.json',
    'test_file': './data/FN/sc_test100/test.jsonn',
    'bert_vocab_file': './model/chinese_wwm_ext/vocab.txt',
    # 'tag_file': './data/ccks/ccks+FN_tags_list.txt',
    'tag_file': './data/FN/tags_list.txt',
    'loader_name': 'ptloader_v2',
    "word_embedding_file": "./data/tencent/word_embedding.txt",
    "word_vocab_file": "./data/tencent/tencent_vocab.txt",
    "word_vocab_file_with_tag": "./data/tencent/tencent_vocab_with_tag.json",
    "default_tag": "O",
    'batch_size': 8,
    'eval_batch_size': 32,
    'pass_none_rule': True,
    'skip_single_matched_word': True,
    'do_shuffle': True,
    'task_name': 'sc_fj_tags_keyword_pretrain_2',
    "use_gpu": True,
    "debug": True,
    "tag_rules": {
        "O": "非实体",
        "KEYWORD":"异常关键词",
        "DIS": "疾病或诊断",
        "ANA": "解剖部位",
        "LAB": "实验室检验",
        "MED": "药物",
        "OPE": "手术",
        "IMA": "影像检查",
    }
    
}

from CC.pre_trained import NERPreTrainer
pre_trainer = NERPreTrainer(**args)

for i in pre_trainer(lr=1e-5):
    a = i

In [ ]:
# 16 lr2=1e-2
#  5.验证 7是有左括号切分 8是没有左括号 9是强制120切分 10在9的基础上120匹配左括号切分 67% 11 keyword 12 lexloader 
#  只有400条的sc数据集做多标签预训练  13 训练集只有四川   14 800条 15 只有keyword 
#  16 只有福建，训练测试验证也是福建。出现了0
#  17 在训练集只有福建
#  FN_fj_sc_tag_ccks_pre_2_eval ccks多标签 测试集100
#  FN_sc_fj_tags_keyword_pretrain_2 只有keyword
from CC.trainer import NERTrainer
args = {
    'num_epochs': 30,
    'num_gpus': [0, 1],
    'bert_config_file_name': './model/chinese_wwm_ext/bert_config.json',
    # 'pretrained_file_name': './model/chinese_wwm_ext/pytorch_model.bin',
    'pretrained_file_name': './save_pretrained/sc_fj_tags_keyword_pretrain_2/Bert_15100/pytorch_model.bin',
    'hidden_dim': 300,
    'max_seq_length': 512,
    'max_scan_num': 1000000,
    # 'train_file': './data/FN/fj+sc/train(400).json',
    # 'train_file': './data/FN/sc_json_500/train.json',
    'train_file': './data/FN/fj+sc/train(400).json',
    'eval_file': './data/FN/sc_test100/test.json',
    'test_file': './data/FN/sc_test100/test.json',
    'bert_vocab_file': './model/chinese_wwm_ext/vocab.txt',
    'tag_file': './data/FN/tags_list.txt',
    'loader_name': 'le_loader',
    'output_eval':True,
    "word_embedding_file":"./data/tencent/word_embedding.txt",
    "word_vocab_file":"./data/tencent/tencent_vocab.txt",
    # "word_vocab_file_with_tag": "./data/tencent/tencent_vocab_with_tag.json",
    "default_tag":"O",
    'batch_size': 8,
    'eval_batch_size': 64,
    'do_shuffle': True,
    "use_gpu": True,
    "debug": True,
    'model_name': 'LEBert',
    'task_name': 'FN_sc_fj_tags_keyword_pretrain_2'
}
trainer = NERTrainer(**args)

for i in trainer(lr2=1e-2):
    a = i

In [ ]:
# 16 lr2=1e-2
#  5.验证 7是有左括号切分 8是没有左括号 9是强制120切分 10在9的基础上120匹配左括号切分 67% 11 keyword 12 lexloader 
#  只有400条的sc数据集做多标签预训练  13 训练集只有四川   14 800条 15 只有keyword
# 
from CC.trainer import NERTrainer
args = {
    'num_epochs': 30,
    'num_gpus': [0, 1],
    'bert_config_file_name': './model/chinese_wwm_ext/bert_config.json',
    # 'pretrained_file_name': './model/chinese_wwm_ext/pytorch_model.bin',
    'pretrained_file_name': './save_pretrained/sc&fj_tags_ccks_pretrain_2/Bert_10088/pytorch_model.bin',
    'hidden_dim': 300,
    'max_seq_length': 512,
    'max_scan_num': 1000000,
    # 'train_file': './data/FN/fj+sc/train(400).json',
    # 'train_file': './data/FN/sc_json_500/train.json',
    'train_file': './data/FN/fj+sc/train(400).json',
    'eval_file': './data/FN/sc_test100/test.json',
    'test_file': './data/FN/sc_test100/test.json',
    'bert_vocab_file': './model/chinese_wwm_ext/vocab.txt',
    'tag_file': './data/FN/tags_list.txt',
    'loader_name': 'le_loader',
    'output_eval':True,
    "word_embedding_file":"./data/tencent/word_embedding.txt",
    "word_vocab_file":"./data/tencent/tencent_vocab.txt",
    # "word_vocab_file_with_tag": "./data/tencent/tencent_vocab_with_tag.json",
    "default_tag":"O",
    'batch_size': 8,
    'eval_batch_size': 64,
    'do_shuffle': True,
    "use_gpu": True,
    "debug": True,
    'model_name': 'LEBert',
    'task_name': 'FN_fj_sc_tag_ccks_pre_2'
}
trainer = NERTrainer(**args)

for i in trainer(lr2=1e-2):
    a = i

#### 转换成json

In [ ]:
from tools.to_json import conll_to_json
conll_to_json('./data/ccks/subtask1_training_part2.txt', './data/ccks/3.csv', split_tag='\n\n')


In [ ]:
from tools.to_json import conll_to_json
conll_to_json('./data/FN/sc-super/dev.csv', './data/FN/sc-super/dev.json', split_tag='\n\n')
conll_to_json('./data/FN/sc-super/test.csv', './data/FN/sc-super/test.json', split_tag='\n\n')
conll_to_json('./data/FN/sc-super/train.csv', './data/FN/sc-super/train.json', split_tag='\n\n')

#### 测试

In [ ]:
import json
filename = "data/FN/fj+sc/train(400).json"
sum = 0
with open(filename, "r", encoding="utf-8") as f:
    for line in f:
        data = json.loads(line)
        text = data["text"]
        label = data["label"]
        
        if len(text)>2000:
            sum = sum+1
print(sum)        

In [ ]:
from CC.loaders import *
from tqdm import *
from CC.loaders.pretrain import *
from CC.loaders.pretrain.ptloader_v2 import *
import json
from CC.predicter import NERPredict
args = {
    'num_epochs': 35,
    'num_gpus': [0, 1, 2, 3],
    'bert_config_file_name': './model/chinese_wwm_ext/bert_config.json',
    # 'pretrained_file_name': './model/chinese_wwm_ext/pytorch_model.bin',
    'pretrained_file_name': './save_pretrained/FN_multiple_pretrained_400_6/Bert_1200/pytorch_model.bin',
    'max_seq_length': 512,
    'max_scan_num': 1000000,
    'train_file': './data/FN/sc-super/train_super_400.json',
    'eval_file': './data/FN/sc-json-500/dev.json',
    'test_file': './data/FN/sc-json-500/test.json',
    'bert_vocab_file': './model/chinese_wwm_ext/vocab.txt',
    'tag_file': 'data/ccks/ccks+FN_tags_list.txt',
    'loader_name': 'ptloader_v2',
    "word_embedding_file": "./data/tencent/word_embedding.txt",
    "word_vocab_file": "./data/tencent/tencent_vocab.txt",
    "word_vocab_file_with_tag": "./data/tencent/tencent_vocab_with_tag.json",
    "default_tag": "O",
    'batch_size': 8,
    'eval_batch_size': 32,
    'pass_none_rule': True,
    'skip_single_matched_word': True,
    'do_shuffle': True,
    'task_name': 'FN_multiple_pretrained_400_6',
    "use_gpu": True,
    "debug": True,
    "tag_rules": {
        "O": "非实体",
        "KEYWORD":"关键词",
        "DIS": "疾病诊断",
        "ANA": "解剖部位",
        "LAB": "实验检验",
        "MED": "药物",
        "OPE": "手术",
        "IMA": "影像检查",
    }
}


loader = PTLoaderV2(**args)


# loader.tag_vocab.id2token(loader.myData[0]["labels"].tolist())

# loader.tokenizer.decode(loader.myData[0]["input_labels"])

In [ ]:
loader.myData[799]["input_ids"]
loader.tokenizer.decode(loader.myData[0]["input_ids"])

In [ ]:
loader.myData[0]["labels"]

#### 重新验证

In [ ]:
# 
from CC.trainer import NERTrainer
args = {
    'num_epochs': 30,
    'num_gpus': [0, 1],
    'bert_config_file_name': './model/chinese_wwm_ext/bert_config.json',
    # 'pretrained_file_name': './model/chinese_wwm_ext/pytorch_model.bin',
    'pretrained_file_name': './save_pretrained/FN/FN_spuer_pretrained_400_5/Bert_12832/pytorch_model.bin',
    'hidden_dim': 300,
    'max_seq_length': 512,
    'max_scan_num': 1000000,
    # 'train_file': './data/FN/fj+sc/train(400).json',
    # 'train_file': './data/FN/sc_json_500/train.json',
    'train_file': './data/FN/fj+sc/train(400).json',
    'eval_file': './data/FN/sc_test100/test.json',
    'test_file': './data/FN/sc_test100/test.json',
    'bert_vocab_file': './model/chinese_wwm_ext/vocab.txt',
    'tag_file': './data/FN/tags_list.txt',
    'loader_name': 'le_loader',
    'output_eval':True,
    "word_embedding_file":"./data/tencent/word_embedding.txt",
    "word_vocab_file":"./data/tencent/tencent_vocab.txt",
    # "word_vocab_file_with_tag": "./data/tencent/tencent_vocab_with_tag.json",
    "default_tag":"O",
    'batch_size': 8,
    'eval_batch_size': 64,
    'do_shuffle': True,
    "use_gpu": True,
    "debug": True,
    'model_name': 'LEBert',
    'task_name': 'FN_sc_test100_pre'
}
trainer = NERTrainer(**args)

for i in trainer(lr2=1e-2):
    a = i

In [ ]:
# 
from CC.trainer import NERTrainer
args = {
    'num_epochs': 30,
    'num_gpus': [0, 1],
    'bert_config_file_name': './model/chinese_wwm_ext/bert_config.json',
    'pretrained_file_name': './model/chinese_wwm_ext/pytorch_model.bin',
    # 'pretrained_file_name': './save_pretrained/FN/FN_spuer_pretrained_400_5/Bert_12832/pytorch_model.bin',
    'hidden_dim': 300,
    'max_seq_length': 512,
    'max_scan_num': 1000000,
    # 'train_file': './data/FN/fj+sc/train(400).json',
    # 'train_file': './data/FN/sc_json_500/train.json',
    'train_file': './data/FN/fj+sc/train(400).json',
    'eval_file': './data/FN/sc_test100/test.json',
    'test_file': './data/FN/sc_test100/test.json',
    'bert_vocab_file': './model/chinese_wwm_ext/vocab.txt',
    'tag_file': './data/FN/tags_list.txt',
    'loader_name': 'le_loader',
    'output_eval':True,
    "word_embedding_file":"./data/tencent/word_embedding.txt",
    "word_vocab_file":"./data/tencent/tencent_vocab.txt",
    # "word_vocab_file_with_tag": "./data/tencent/tencent_vocab_with_tag.json",
    "default_tag":"O",
    'batch_size': 4,
    'eval_batch_size': 64,
    'do_shuffle': True,
    "use_gpu": True,
    "debug": True,
    'model_name': 'LEBert',
    'task_name': 'FN_sc_test100_bs_4'
}
trainer = NERTrainer(**args)

for i in trainer():
    a = i

In [ ]:
# 
from CC.trainer import NERTrainer
args = {
    'num_epochs': 30,
    'num_gpus': [0, 1],
    'bert_config_file_name': './model/chinese_wwm_ext/bert_config.json',
    'pretrained_file_name': './model/chinese_wwm_ext/pytorch_model.bin',
    # 'pretrained_file_name': './save_pretrained/FN_spuer_pretrained_fj_pt/Bert_21700/pytorch_model.bin',
    'hidden_dim': 300,
    'max_seq_length': 512,
    'max_scan_num': 1000000,
    # 'train_file': './data/FN/fj+sc/train(400).json',
    # 'train_file': './data/FN/sc_json_500/train.json',
    'train_file': './data/FN/fj+sc/train(400).json',
    'eval_file': './data/FN/fj+sc/test(100+100).json',
    'test_file': './data/FN/fj+sc/test(100+100).json',
    'bert_vocab_file': './model/chinese_wwm_ext/vocab.txt',
    'tag_file': './data/FN/tags_list.txt',
    'loader_name': 'le_loader',
    'output_eval':True,
    "word_embedding_file":"./data/tencent/word_embedding.txt",
    "word_vocab_file":"./data/tencent/tencent_vocab.txt",
    # "word_vocab_file_with_tag": "./data/tencent/tencent_vocab_with_tag.json",
    "default_tag":"O",
    'batch_size': 8,
    'eval_batch_size': 64,
    'do_shuffle': True,
    "use_gpu": True,
    "debug": True,
    'model_name': 'LEBert',
    'task_name': 'FN_sc_fj_test200'
}
trainer = NERTrainer(**args)

for i in trainer(lr2=1e-2):
    a = i

#### 验证仅四川的和福建的（ccks标签&keyword标签）

In [ ]:
# 2.预测 
from CC.trainer import NERTrainer
args = {
    'num_epochs': 30,
    'num_gpus': [0, 1],
    'bert_config_file_name': './model/chinese_wwm_ext/bert_config.json',
    'pretrained_file_name': './model/chinese_wwm_ext/pytorch_model.bin',
    # 'pretrained_file_name': './save_pretrained/ccks_pretrained_2/Bert_6055/pytorch_model.bin', 
    'hidden_dim': 300,
    'max_seq_length': 512,
    'max_scan_num': 1000000,
    'train_file': './data/ccks/train.json',
    'eval_file': './data/ccks/dev.json',
    'test_file': './data/ccks/test.json',
    'bert_vocab_file': './model/chinese_wwm_ext/vocab.txt',
    'tag_file': 'data/ccks/ccks+FN_tags_list.txt',
    'loader_name': 'le_loader',
    'output_eval':True,
    "word_embedding_file":"./data/tencent/word_embedding.txt",
    "word_vocab_file":"./data/tencent/tencent_vocab.txt",
    "default_tag":"O",
    'batch_size': 16,
    'eval_batch_size': 64,
    'do_shuffle': True,
    "use_gpu": True,
    "debug": True,
    'model_name': 'LEBert',
    'task_name': 'ccks_predict_model_2'
}
# trainer = NERTrainer(**args)

# for i in trainer():
#     a = i

In [ ]:
# 拓展数据集
from CC.predicter import NERPredict
import json

# 使用了预训练模型
args["lstm_crf_model_file"] = "save_model/ccks_predict_model_2/lstm_crf/lstm_crf_690.pth"
args["bert_model_file"] = "save_model/ccks_predict_model_2/LEBert/LEBert_690.pth"
predict = NERPredict(**args)

filename = "data/FN/sc_json_500/pretrain_keyword.json"

batch_size = 40
index = 0
sentences = []

with open("data/FN/sc_json_500/pretrain_ccks.json", "w", encoding="utf-8") as out:
    with open(filename, "r", encoding="utf-8") as f:
        for line in f:
            data = json.loads(line)
            text = data["text"]
            
            sentences.append(text)
            index += 1
            if index % batch_size == batch_size-1:
                for s, label in predict(sentences):
                    
                    assert len(s[:args["max_seq_length"]-2])==len(label)
                    out.write(f"""{json.dumps({"text":s[:args["max_seq_length"]-2],"label":label},ensure_ascii=False)}\n""")
                sentences = []
                out.flush()
        if len(sentences)>0:
            for s, label in predict(sentences):
                assert len(s[:args["max_seq_length"]])==len(label)
                out.write(f"""{json.dumps({"text":s[:args["max_seq_length"]-2],"label":label},ensure_ascii=False)}\n""")

In [ ]:
# 预训练
# sc_ccks_pretrain_3:sc的ccks预训练 13600
# sc_keyword_pretrain_2:sc的keyword预训练 34000
# fj_keyword_pretrain_2 27200
# fj_ccks_pretrain_2 10940
args = {
    'num_epochs': 35,
    'num_gpus': [0, 1],
    'bert_config_file_name': './model/chinese_wwm_ext/bert_config.json',
    # 'pretrained_file_name': './model/chinese_wwm_ext/pytorch_model.bin',
    'pretrained_file_name': './save_pretrained/fj_ccks_pretrain_1/Bert_4080/pytorch_model.bin',
    'max_seq_length': 512,
    'max_scan_num': 1000000,
    'train_file': './data/FN/fj_json/pretrain_keyword.json',
    'eval_file': './data/FN/fj_json/eval.json',
    'test_file': './data/FN/fj_json/test.jsonn',
    'bert_vocab_file': './model/chinese_wwm_ext/vocab.txt',
    'tag_file': './data/ccks/ccks+FN_tags_list.txt',
    # 'tag_file': './data/FN/tags_list.txt',
    'loader_name': 'ptloader_v2',
    "word_embedding_file": "./data/tencent/word_embedding.txt",
    "word_vocab_file": "./data/tencent/tencent_vocab.txt",
    "word_vocab_file_with_tag": "./data/tencent/tencent_vocab_with_tag.json",
    "default_tag": "O",
    'batch_size': 8,
    'eval_batch_size': 32,
    'pass_none_rule': True,
    'skip_single_matched_word': True,
    'do_shuffle': True,
    'task_name': 'fj_ccks_pretrain_2',
    "use_gpu": True,
    "debug": True,
    "tag_rules": {
        "O": "非实体",
        "KEYWORD":"异常关键词",
        "DIS": "疾病或诊断",
        "ANA": "解剖部位",
        "LAB": "实验室检验",
        "MED": "药物",
        "OPE": "手术",
        "IMA": "影像检查",
    }
    
}

from CC.pre_trained import NERPreTrainer
pre_trainer = NERPreTrainer(**args)

for i in pre_trainer(lr=1e-5):
    a = i

## 验证

In [3]:
from CC.trainer import NERTrainer
args = {
    'num_epochs': 30,
    'num_gpus': [0, 1],
    'bert_config_file_name': './model/chinese_wwm_ext/bert_config.json',
    # 'pretrained_file_name': './model/chinese_wwm_ext/pytorch_model.bin',
    # 'pretrained_file_name': './save_pretrained/sc_ccks_pretrain_3/Bert_13600/pytorch_model.bin',
    # 'pretrained_file_name': './save_pretrained/sc_keyword_pretrain_2/Bert_34000/pytorch_model.bin',
    'hidden_dim': 300,
    'max_seq_length': 512,
    'max_scan_num': 1000000,
    # 'train_file': './data/FN/fj+sc/train(400).json',
    'train_file': './data/FN/sc_json_500/train.json',
    'eval_file': './data/FN/sc_test100/test.json',
    'test_file': './data/FN/sc_test100/test.json',
    'bert_vocab_file': './model/chinese_wwm_ext/vocab.txt',
    'tag_file': './data/FN/tags_list.txt',
    'loader_name': 'le_loader',
    'output_eval':True,
    "word_embedding_file":"./data/tencent/word_embedding.txt",
    # "word_vocab_file":"./data/tencent/tencent_vocab.txt",
    # "word_vocab_file":"./data/FN/medicine_vocab.txt",
    # "word_vocab_file":"./data/tencent/tencent_medicine_vocab.txt",
    # "word_vocab_file_with_tag": "./data/tencent/tencent_vocab_with_tag.json",
    "default_tag":"O",
    'batch_size': 8,
    'eval_batch_size': 64,
    'do_shuffle': True,
    "use_gpu": True,
    "debug": True,
    'model_name': 'LEBert',
    # 'task_name': 'sc_medicine_vocab_baseline_4'
}
# trainer = NERTrainer(**args)

# for i in trainer(lr2=1e-2):
#     a = i

In [9]:
#四川
# sc_400_baseline:0.6795、0.6949、0.6936 0.6868 0.6804
# sc_ccks_dev_1 0.6831 0.6833 0.6804
# sc_keyword_dev_1 预训练和文件的标签keyword 0.6868 0.6936 0.6788

# sc_medicine_vocab_baseline 0.6802 0.6882 0.6824 0.7127 0.6702 0.6872
# sc_medicine_vocab_pretrain_ccks 0.6647
# sc_medicine_vocab_pretrain_keyword 0.6890

# sc_tx_med_baseline 0.6820 0.6838 0.6917 0.6936 0.68

# args["pretrained_file_name"] = "./model/chinese_wwm_ext/pytorch_model.bin"
args["pretrained_file_name"] = "./save_pretrained/sc_ccks_pretrain_3/Bert_13600/pytorch_model.bin"
# args["pretrained_file_name"] = "./save_pretrained/sc_keyword_pretrain_2/Bert_34000/pytorch_model.bin"

# args["word_vocab_file"] ="./data/tencent/tencent_vocab.txt"
args["word_vocab_file"] ="./data/FN/medicine_vocab.txt"
# args["word_vocab_file"] ="./data/tencent/tencent_medicine_vocab.txt"

args["task_name"] = "sc_medicine_pretrain_ccks_2"

trainer = NERTrainer(**args)

for i in trainer(lr2=1e-2):
    a = i

kwargs parser: {
    "batch_size": 8,
    "eval_batch_size": 64,
    "test_batch_size": 16,
    "word_embedding_file": "./data/tencent/word_embedding.txt",
    "word_vocab_file": "./data/FN/medicine_vocab.txt",
    "train_file": "./data/FN/sc_json_500/train.json",
    "eval_file": "./data/FN/sc_test100/test.json",
    "test_file": "./data/FN/sc_test100/test.json",
    "tag_file": "./data/FN/tags_list.txt",
    "bert_vocab_file": "./model/chinese_wwm_ext/vocab.txt",
    "output_eval": true,
    "max_scan_num": 1000000,
    "add_seq_vocab": false,
    "max_seq_length": 512,
    "max_word_num": 5,
    "default_tag": "O",
    "use_test": false,
    "do_shuffle": true,
    "do_predict": false,
    "task_name": "sc_medicine_pretrain_ccks_2"
}


calculate ./data/FN/sc_json_500/train.json etag: 100%|██████████| 4.71M/4.71M [00:00<00:00, 364MB/s]
calculate ./data/FN/sc_test100/test.json etag: 100%|██████████| 1.18M/1.18M [00:00<00:00, 371MB/s]
calculate ./data/FN/sc_test100/test.json etag: 100%|██████████| 1.18M/1.18M [00:00<00:00, 364MB/s]
calculate ./data/FN/tags_list.txt etag: 100%|██████████| 21.0/21.0 [00:00<00:00, 62.7kB/s]

load cached ./temp/88ff756ec83be36f241527dc9bf49e5d_2bee60a32e19544d149be8c360d1a109_2bee60a32e19544d149be8c360d1a109_b43c9a3dc281dcbcfc242873c520c19f/1000000/lexicon_tree


load cached ./temp/88ff756ec83be36f241527dc9bf49e5d_2bee60a32e19544d149be8c360d1a109_2bee60a32e19544d149be8c360d1a109_b43c9a3dc281dcbcfc242873c520c19f/1000000/matched_words
load cached ./temp/88ff756ec83be36f241527dc9bf49e5d_2bee60a32e19544d149be8c360d1a109_2bee60a32e19544d149be8c360d1a109_b43c9a3dc281dcbcfc242873c520c19f/1000000/word_vocab


count line size ./data/FN/tags_list.txt: 3L [00:00, 21112.27L/s]
build line mapper: 3L [00:00, 5361.27L/s]/3 [00:00<?, ?it/s]
load vocab from list: 100%|██████████| 3/3 [00:00<00:00, 47662.55it/s]


load cached ./temp/88ff756ec83be36f241527dc9bf49e5d_2bee60a32e19544d149be8c360d1a109_2bee60a32e19544d149be8c360d1a109_b43c9a3dc281dcbcfc242873c520c19f/1000000/vocab_embedding


load dataset from ./data/FN/sc_json_500/train.json: 100%|██████████| 400/400 [00:02<00:00, 153.36it/s]
load dataset from ./data/FN/sc_test100/test.json: 100%|██████████| 100/100 [00:00<00:00, 150.34it/s]
Some weights of the model checkpoint at ./save_pretrained/sc_ccks_pretrain_3/Bert_13600/pytorch_model.bin were not used when initializing LEBertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing LEBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LEBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertFo

Load pretrained embedding from file.........


Eval Result: 100%|██████████| 2/2 [00:01<00:00,  1.01it/s, F1=0.676, eval_acc=0.957, eval_loss=27, eval_precision=0.679, eval_recall=0.674] 


In [ ]:
# 福建

from CC.trainer import NERTrainer
args = {
    'num_epochs': 30,
    'num_gpus': [0, 1],
    'bert_config_file_name': './model/chinese_wwm_ext/bert_config.json',
    # 'pretrained_file_name': './model/chinese_wwm_ext/pytorch_model.bin',
    # 'pretrained_file_name': './save_pretrained/fj_ccks_pretrain_2/Bert_19040/pytorch_model.bin',
    'pretrained_file_name': './save_pretrained/fj_keyword_pretrain_2/Bert_27200/pytorch_model.bin',
    'hidden_dim': 300,
    'max_seq_length': 512,
    'max_scan_num': 1000000,
    # 'train_file': './data/FN/fj+sc/train(400).json',
    'train_file': './data/FN/fj_json/train_400.json',
    'eval_file': './data/FN/fj_json/dev.json',
    'test_file': './data/FN/fj_json/test.json',
    'bert_vocab_file': './model/chinese_wwm_ext/vocab.txt',
    'tag_file': './data/FN/tags_list.txt',
    'loader_name': 'le_loader',
    'output_eval':True,
    "word_embedding_file":"./data/tencent/word_embedding.txt",
    # "word_vocab_file":"./data/tencent/tencent_vocab.txt",
    "word_vocab_file":"./data/FN/medicine_vocab.txt",
    # "word_vocab_file_with_tag": "./data/tencent/tencent_vocab_with_tag.json",
    "default_tag":"O",
    'batch_size': 8,
    'eval_batch_size': 64,
    'do_shuffle': True,
    "use_gpu": True,
    "debug": True,
    'model_name': 'LEBert',
    # 'task_name': 'fj_medicine_vocab_keyword'
}
# trainer = NERTrainer(**args)

# for i in trainer(lr2=1e-2):
#     a = i

In [ ]:
# 福建
# fj_baseline 0.9233
# fj_keyword_dev_1 0.9039
# fj_ccks_dev_1 0.9219

# fj_medicine_vocab_baseline 0.9289
# fj_medicine_vocab_ccks 0.9220
# fj_medicine_vocab_keyword 0.9079

args["pretrained_file_name"] = "./model/chinese_wwm_ext/pytorch_model.bin"
# args["pretrained_file_name"] = "./save_pretrained/fj_ccks_pretrain_2/Bert_19040/pytorch_model.binn"
# args["pretrained_file_name"] = "./save_pretrained/fj_keyword_pretrain_2/Bert_27200/pytorch_model.bin"

args["word_vocab_file"] ="./data/tencent/tencent_vocab.txt"
# args["word_vocab_file"] ="./data/FN/medicine_vocab.txt"
# args["word_vocab_file"] ="./data/tencent/tencent_medicine_vocab.txt"

args["task_name"] = "fj_baseline"

trainer = NERTrainer(**args)

for i in trainer(lr2=1e-2):
    a = i

In [ ]:
# ccks
# ccks_baseline 0.8258 0.8197 
# ccks_medicine_vocab_baseline 0.8149 0.8120
from CC.trainer import NERTrainer

args = {
    'num_epochs': 30,
    'num_gpus': [0, 1, 2, 3],
    'bert_config_file_name': './model/chinese_wwm_ext/bert_config.json',
    'pretrained_file_name': './model/chinese_wwm_ext/pytorch_model.bin',
    'hidden_dim': 300,
    'max_seq_length': 150,
    'max_scan_num': 1000000,
    'train_file': './data/ccks/train.json',
    'eval_file': './data/ccks/dev.json',
    'test_file': './data/ccks/test.json',
    'bert_vocab_file': './model/chinese_wwm_ext/vocab.txt',
    'tag_file': 'data/ccks/ccks_tags_list.txt',
    'output_eval': True,
    'loader_name': 'le_loader',
    "word_embedding_file": "./data/tencent/word_embedding.txt",
    # "word_vocab_file": "./data/tencent/tencent_vocab.txt",
    "word_vocab_file":"./data/FN/medicine_vocab.txt0",
    "default_tag": "O",
    'batch_size': 32,
    'eval_batch_size': 64,
    'do_shuffle': True,
    'model_name': 'LEBert',
    'task_name': 'ccks_medicine_vocab_baseline_2'
}

# Trainer
trainer = NERTrainer(**args)

for i in trainer(lr2=1e-2):
    a = i